In [1]:
# app.py
from flask import Flask, request, render_template, flash, send_file,jsonify, send_from_directory
import numpy as np
from PIL import Image
import torch
from diffusers import StableDiffusionPipeline
import secrets


e:\ProgramsInstallation\anaconda3\envs\tf_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

True
NVIDIA GeForce RTX 2070 with Max-Q Design


In [3]:
model = "dreamlike-art/dreamlike-diffusion-1.0"

pipe = StableDiffusionPipeline.from_pretrained(model, torch_dtype=torch.float16, use_safetensors=True)
pipe = pipe.to("cuda")

Loading pipeline components...: 100%|██████████| 5/5 [00:03<00:00,  1.45it/s]


In [4]:
#torch.save(pipe, "E:/CodePractice/Flask/model.h5")

In [5]:
try:
    # Load the Keras model
    model = torch.load("E:/CodePractice/Flask/Final API/model.h5")
    print("Model loaded successfully.")
except Exception as e:
    print("Error loading model:", e)


Error loading model: [Errno 2] No such file or directory: 'E:/CodePractice/Flask/Final API/model.h5'


'\nfrom flask import Flask, request, send_file\nfrom PIL import Image, ImageDraw, ImageFont\nimport io\n\napp = Flask(__name__)\n\n@app.route(\'/generate_image\', methods=[\'POST\'])\ndef generate_image():\n    text = request.json.get(\'text\')\n    if not text:\n        return "Text is required", 400\n    \n    # Create an image with PIL\n    image = Image.new(\'RGB\', (200, 100), color = (73, 109, 137))\n    d = ImageDraw.Draw(image)\n    fnt = ImageFont.load_default()\n    d.text((10,10), text, font=fnt, fill=(255, 255, 0))\n    \n    # Save the image to a bytes buffer\n    img_io = io.BytesIO()\n    image.save(img_io, \'PNG\')\n    img_io.seek(0)\n\n    return send_file(img_io, mimetype=\'image/png\')\n\napp.run()\n'

In [7]:

app = Flask(__name__)

def generate_image(prompt):
    image = np.array(pipe(prompt=prompt).images[0])
    return image

@app.route('/generate', methods=['POST'])
def generate():
    text = request.json['text']
    image = generate_image(text)

    # Convert the NumPy array to a PIL Image
    pil_image = Image.fromarray(image)

    # Save the PIL Image as a temporary file
    random_string = secrets.token_hex(8)

    filename = f'generated_{random_string}.png'
    pil_image.save(filename)

    # Generate the URL for the image
    url = request.host_url + 'static/' + filename
    
    # Return the URL as a JSON response
    return jsonify({'image_url': url})

@app.route('/static/<filename>')
def serve_image(filename):
    return send_from_directory('.', filename)

app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
100%|██████████| 50/50 [00:37<00:00,  1.34it/s]
127.0.0.1 - - [15/Jul/2024 21:18:15] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2024 21:18:16] "GET /static/generated_1ad218fdf7cf996b.png HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2024 21:18:39] "POST /static/generated_1ad218fdf7cf996b.png HTTP/1.1" 405 -
127.0.0.1 - - [15/Jul/2024 21:18:47] "GET /static/generated_1ad218fdf7cf996b.png HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2024 21:18:49] "GET /favicon.ico HTTP/1.1" 404 -
